<a href="https://colab.research.google.com/github/ritamsarkar2000/ML/blob/master/text_classification_with_tf_hub_%2C_learned_from_tensorflow(dot)org.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing required libraries

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np

!pip install tensorflow-hub
!pip install tfds-nightly

In [0]:
#downloading the data

train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised= True)

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [0]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [0]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [0]:
#building the model

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hublayer=hub.KerasLayer(embedding, input_shape=[],
                        dtype= tf.string, trainable= True)

model= tf.keras.Sequential([
                            hublayer,
                           tf.keras.layers.Dense(16, activation='relu'),
                           tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_8 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer= 'adam',
          loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
          metrics= [ 'accuracy' ])

In [0]:
model.fit(train_data.shuffle(10000).batch(512),
          epochs=20,
          validation_data= validation_data.batch(512),
          verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 97ms/step - loss: 0.6818 - accuracy: 0.5298 - val_loss: 0.6693 - val_accuracy: 0.6049
Epoch 2/20
30/30 [==============================] - 3s 96ms/step - loss: 0.6579 - accuracy: 0.6427 - val_loss: 0.6489 - val_accuracy: 0.6738
Epoch 3/20
30/30 [==============================] - 3s 96ms/step - loss: 0.6406 - accuracy: 0.6951 - val_loss: 0.6348 - val_accuracy: 0.6964
Epoch 4/20
30/30 [==============================] - 3s 97ms/step - loss: 0.6265 - accuracy: 0.7383 - val_loss: 0.6230 - val_accuracy: 0.7463
Epoch 5/20
30/30 [==============================] - 3s 97ms/step - loss: 0.6141 - accuracy: 0.7714 - val_loss: 0.6131 - val_accuracy: 0.7746
Epoch 6/20
30/30 [==============================] - 3s 97ms/step - loss: 0.6032 - accuracy: 0.7988 - val_loss: 0.6053 - val_accuracy: 0.7911
Epoch 7/20
30/30 [==============================] - 3s 96ms/step - loss: 0.5938 - accuracy: 0.8211 - val_loss: 0.5986 - val_accuracy: 0.7974
Epoch 8/20
30

In [0]:
test_loss, test_acc = model.evaluate(test_data.batch(512), verbose=1)
print("the accuracy is", test_acc)
print("the loss",  test_loss)

49/49 [==============================] - 3s 64ms/step - loss: 0.5726 - accuracy: 0.8560
the accuracy is 0.8559600114822388
the loss 0.572597324848175


***Closer to 0 means negetive and closer to 1 means positive***

In [0]:
print(model.predict(["this is the best movie"]))

[[0.9805962]]


In [0]:
print(model.predict(["world's worst and the most disgusting movie"]))

[[0.00694677]]
